In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm

data = pd.read_csv("DATA/Standard_Scaler_result.csv",encoding="cp949")
data.head()

,CUSTNO,GNO,CBSCORE,CBSCOREGRD,CREDITOTAMT,YSALEAMT,ESTMM,ASSETAMT,IMSAAMT,IMJUAMT,...,환율,GDP,소상공인체감지수,실업률,물가지수,국고채,금리,유가등락률,소비자심리지수,ONEHOT
0,475821,l180202101898,1.483693,-1.008776,-0.182324,5.599410,-0.351065,-0.497246,-0.176818,-0.327516,...,-0.068090,2.274533,-1.302456,-0.290076,2.728294,-1.492204,-3.801670,2.712699,1.300530,3
1,74417,l230201700120,0.436620,-0.269293,-0.187359,3.004204,-0.774902,-0.434147,-0.395499,0.047573,...,0.509001,-1.236236,-1.058419,0.571922,-0.733929,-0.363940,-0.649856,1.705658,-1.167094,8
2,387787,l110201603233,0.682990,-1.008776,-0.283032,1.765241,1.889216,-0.497246,-0.176818,-0.327516,...,0.991659,-1.589789,0.853200,-0.290076,-1.872513,-0.546902,-0.649856,0.170943,-0.965339,7
3,395418,l230201700254,0.169719,-0.269293,-0.212536,1.647960,-0.744628,-0.497246,-0.176818,-0.327516,...,0.490639,-1.236236,1.219254,-0.721075,-0.788147,-0.241965,-0.649856,0.665173,-0.810142,3
4,190372,l200201601418,-3.033092,2.688638,-0.247784,0.478163,1.616749,-0.202785,-0.351763,-0.327516,...,0.409321,-1.809655,-0.102610,-0.721075,-2.530877,-1.736154,-0.649856,-0.958727,-0.142797,7


In [43]:
features = data[['CBSCORE', 'CREDITOTAMT', 'YSALEAMT',
       'ESTMM',  'IMSAAMT', 'IMJUAMT', 'BUSAAMT', 'BUJUAMT',
       'BU1TOTAMT', 'GAMT', 'LABORCNT', 'KOSPI', '환율', 'GDP', 
       '소상공인체감지수', '실업률', '물가지수', '국고채', '금리', '유가등락률',
       '소비자심리지수','ONEHOT']]
act = data['ACTCD']

In [44]:
act = act - 1
act = act * -1

In [45]:
from sklearn.model_selection import train_test_split

train_x, test_x,train_y, test_y = train_test_split(features, act,
stratify=act,train_size=0.7,test_size=0.3,random_state=1)

print(train_x.shape, test_x.shape, train_y.shape, test_y.shape)

(27381, 22) (11736, 22) (27381,) (11736,)


In [46]:
from imblearn.over_sampling import BorderlineSMOTE, ADASYN
from imblearn.under_sampling import EditedNearestNeighbours, TomekLinks
from imblearn.combine import SMOTEENN, SMOTETomek

#언더 샘플링
enn = EditedNearestNeighbours(kind_sel="all", n_neighbors=10)
tomekl = TomekLinks()

#오버 샘플링
bsmote = BorderlineSMOTE(random_state=42)
adasyn = ADASYN(random_state=42)

#혼합 샘플링
smotee = SMOTEENN(random_state=42)
smoteT = SMOTETomek(random_state=42)


X_under1_train, Y_under1_train = enn.fit_resample(train_x, train_y)
X_under2_train, Y_under2_train = tomekl.fit_resample(train_x, train_y)

X_over1_train, Y_over1_train = bsmote.fit_resample(train_x,train_y)
X_over2_train, Y_over2_train = adasyn.fit_resample(train_x,train_y)

X_comb1_train, Y_comb1_train = smotee.fit_resample(train_x, train_y)
X_comb2_train, Y_comb2_train = smoteT.fit_resample(train_x, train_y)

In [5]:
def cut_off(y, threshold) :
    Y = y.copy()  # 대문자 Y를 새로운 변수로 하여 기존의 y값에 영향이 가지 않도록 한다.
    Y[Y>threshold] = 1
    Y[Y<threshold] = 0
    return Y.astype(int)

from sklearn.metrics import confusion_matrix
# confusion matrix accuracy(정확도) 계산함수

def acc(cfmat):
    return (cfmat[0,0] + cfmat[1,1])/(cfmat[0,0] + cfmat[1,1] + cfmat[0,1] + cfmat[1,0])

# 오버샘플링

1. BorderlineSMOTE

In [6]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

In [7]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)

In [8]:
from sklearn.model_selection import cross_val_score
cross_val_score(eclf, X_over1_train, Y_over1_train, cv=5).mean()

0.8536668675909181

In [9]:
from sklearn.tree import DecisionTreeClassifier
DecisionTreeClassifier()

DecisionTreeClassifier()

In [10]:
AdaBoostClassifier()

AdaBoostClassifier()

In [11]:
params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

In [12]:
from sklearn.model_selection import GridSearchCV
grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over1_train, Y_over1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 26}

In [68]:
grid.best_estimator_.feature_importances_

array([0.06076015, 0.04757833, 0.01397005, 0.06923966, 0.11077184,
       0.08609322, 0.00898922, 0.00955078, 0.03200731, 0.13361889,
       0.07681563, 0.03462614, 0.        , 0.02970873, 0.        ,
       0.        , 0.01991415, 0.01861974, 0.00448064, 0.02110397,
       0.01310338, 0.20904817])

In [47]:
from sklearn.metrics import precision_score,recall_score,f1_score,roc_auc_score

dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 26)
model = abc.fit(X_over1_train, Y_over1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [48]:
from sklearn.metrics import classification_report
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.19      0.52      0.28      1070
        정상보증       0.94      0.78      0.85     10666

    accuracy                           0.75     11736
   macro avg       0.57      0.65      0.56     11736
weighted avg       0.87      0.75      0.80     11736



In [49]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[ 560,  510],
       [2389, 8277]], dtype=int64)

In [71]:
accuracy = acc(cfmat)
accuracy

0.7430981595092024

In [72]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.18229336818033323
recall:  0.5214953271028038
f1 score:  0.2701525054466231
accuracy:  0.6434122050852478


2. ADASYN

In [17]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_over2_train, Y_over2_train, cv=5).mean()

0.8494508119337075

In [18]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_over2_train, Y_over2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.55,
 'n_estimators': 27}

In [50]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.55, n_estimators = 27)
model = abc.fit(X_over2_train, Y_over2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [20]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[8100, 2566],
       [ 559,  511]], dtype=int64)

In [51]:
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.18      0.50      0.26      1070
        정상보증       0.94      0.77      0.84     10666

    accuracy                           0.74     11736
   macro avg       0.56      0.63      0.55     11736
weighted avg       0.87      0.74      0.79     11736



In [77]:
accuracy = acc(cfmat)
accuracy

0.7339809134287661

In [78]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.16623292127521144
recall:  0.47757009345794393
f1 score:  0.2466216216216216
accuracy:  0.6186369124705808


# 언더샘플링

1. EditedNearestNeighbours

In [22]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under1_train, Y_under1_train, cv=5).mean()

0.8716165795188475

In [23]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under1_train, Y_under1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'gini',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.45,
 'n_estimators': 25}

In [52]:
dtc = DecisionTreeClassifier(criterion = 'gini', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.45, n_estimators = 25)
model = abc.fit(X_under1_train, Y_under1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [53]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[ 326,  744],
       [ 943, 9723]], dtype=int64)

In [54]:
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.26      0.30      0.28      1070
        정상보증       0.93      0.91      0.92     10666

    accuracy                           0.86     11736
   macro avg       0.59      0.61      0.60     11736
weighted avg       0.87      0.86      0.86     11736



In [83]:
accuracy = acc(cfmat)
accuracy

0.8519938650306749

In [84]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.26161543959971406
recall:  0.34205607476635513
f1 score:  0.29647630619684084
accuracy:  0.6226031358268304


2. TomekLinks

In [27]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_under2_train, Y_under2_train, cv=5).mean()

0.9065857945265081

In [28]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_under2_train, Y_under2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [29]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 27)
model = abc.fit(X_under2_train, Y_under2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [30]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[10666,     0],
       [ 1069,     1]], dtype=int64)

In [55]:
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.26      0.30      0.28      1070
        정상보증       0.93      0.91      0.92     10666

    accuracy                           0.86     11736
   macro avg       0.59      0.61      0.60     11736
weighted avg       0.87      0.86      0.86     11736



In [89]:
accuracy = acc(cfmat)
accuracy

0.9088275391956373

In [90]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.0
recall:  0.0
f1 score:  0.0
accuracy:  0.5


C:\Users\user\anaconda3\envs\Study_Big\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 혼합샘플링

1. SMOTEENN

In [32]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb1_train, Y_comb1_train, cv=5).mean()

0.860899871753066

In [33]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb1_train, Y_comb1_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 26}

In [56]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 26)
model = abc.fit(X_comb1_train, Y_comb1_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [57]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[ 770,  300],
       [3962, 6704]], dtype=int64)

In [58]:
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.16      0.72      0.27      1070
        정상보증       0.96      0.63      0.76     10666

    accuracy                           0.64     11736
   macro avg       0.56      0.67      0.51     11736
weighted avg       0.88      0.64      0.71     11736



In [95]:
accuracy = acc(cfmat)
accuracy

0.6244035446489434

In [96]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.1589701675521046
recall:  0.7271028037383177
f1 score:  0.2608987256874581
accuracy:  0.6706018425216996


2. SMOTETomek

In [37]:
eclf = AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=2), n_estimators=100, learning_rate=0.1)
cross_val_score(eclf, X_comb2_train, Y_comb2_train, cv=5).mean()

0.8502518637920613

In [38]:
DecisionTreeClassifier()
AdaBoostClassifier()

params = {"base_estimator__criterion" : ["gini", "entropy"],
          "base_estimator__max_features" : [7,8,],
          "base_estimator__max_depth" : [1,2],
          "n_estimators": [23,24, 25, 26, 27],
          "learning_rate": [0.4, 0.45, 0.5, 0.55, 0.6]
         }

grid = GridSearchCV(estimator=eclf, param_grid=params, cv=5, n_jobs=7)
grid = grid.fit(X_comb2_train, Y_comb2_train)

grid.best_score_
grid.best_params_

{'base_estimator__criterion': 'entropy',
 'base_estimator__max_depth': 2,
 'base_estimator__max_features': 8,
 'learning_rate': 0.6,
 'n_estimators': 27}

In [59]:
dtc = DecisionTreeClassifier(criterion = 'entropy', max_depth = 2, max_features = 8)
abc = AdaBoostClassifier(learning_rate = 0.6, n_estimators = 27)
model = abc.fit(X_comb2_train, Y_comb2_train)

pred_y = model.predict(test_x)
pred_Y = cut_off(pred_y, 0.5)

In [60]:
cfmat = confusion_matrix(test_y,pred_Y)
cfmat

array([[ 565,  505],
       [2475, 8191]], dtype=int64)

In [61]:
print(classification_report(test_y, pred_Y, target_names=['사고보증', '정상보증']))

              precision    recall  f1-score   support

        사고보증       0.19      0.53      0.27      1070
        정상보증       0.94      0.77      0.85     10666

    accuracy                           0.75     11736
   macro avg       0.56      0.65      0.56     11736
weighted avg       0.87      0.75      0.79     11736



In [101]:
accuracy = acc(cfmat)
accuracy

0.7366223585548739

In [102]:
pr = precision_score(test_y,pred_Y)
r = recall_score(test_y,pred_Y)
f1= f1_score(test_y,pred_Y)
auc_score = roc_auc_score(test_y,pred_Y)

print("precision: ", pr)
print("recall: ", r)
print("f1 score: ", f1)
print("accuracy: ", auc_score)

precision:  0.1694471704285247
recall:  0.48411214953271026
f1 score:  0.2510298037315241
accuracy:  0.623033010824859
